In [1]:
import sourc_code.utils.box_utils
from lib_all import *

In [ ]:
class MultiBoxLoss(nn.Module):
    def __init__(self, jaccard_threshold = 0.5, scale_ne_po = 3, device = "cpu"):
        """
        :param jaccard_threshold: nguong jaccard, lon hon thi la positive
        :param scale_ne_po: ti le negative/positive
        :param device: train tren thiet bi nao
        """
        super(MultiBoxLoss, self).__init__() # khoi tao Module va ke thua
        self.jaccard_threshold = jaccard_threshold
        self.scale_ne_po = scale_ne_po
        self.device = device

    def forward(self, predictions, targets):
        """
        :param prediction: du doan
        :param target: label
        :return:
        """
        loc_data, conf_data, dbox_list = predictions

        #loc_data = (batch_size, num_dbox, 4)
        batch_size = loc_data.size(0)
        num_dbox = loc_data.size(1)
        num_classes = conf_data.size(2)

        # Bien doi thong tin annotation thanh label can hoc bang cach tao ra 2 tensor rong
        conf_t_label = torch.LongTensor(batch_size, num_dbox).to(self.device) # chuyen ve dang long de sau nay dua vao F.
        loc_t= torch.Tensor(batch_size, num_dbox, 4)

        for index in range(batch_size):
            truths_box = targets[index][:, :-1].to(self.device) #(object, location)
            labels = targets[index][:, -1].to(self.device)

            dbox = dbox_list.to(self.device)
            variances = [0.1, 0.2] # he so de bien doi tu dbox sang bbox
            box_utils.match(self.jaccard_threshold, truths_box, dbox, variances, labels, loc_t, conf_t_label, index)